In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "../../scripts/methods/")
from my_paste2 import paste2_align, paste2_align_ref
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

2024-07-05 06:44:01.884503: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-05 06:44:02.584945: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-05 06:44:02.585029: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-05 06:44:02.585036: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT li

In [1]:
method = "PASTE2"
data_folder = "/home/ylu/project/MOSTA/data/"
results_folder = f"./results/{method}/"
figures_folder = f"./results/figures/{method}"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [2]:
## load the data
from tqdm import tqdm
slices = []
for i in tqdm(range(1, 14)):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f"E16.5_E2S{i}.MOSTA.h5ad")))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [01:13<00:00,  5.67s/it]


In [3]:
spatial_key = "spatial"

In [4]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [5]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"

In [ ]:
## Perform the Moscot alignment
sampling_num = 5000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    time_start = time.time()
    align_slices, pis = paste2_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        verbose=True,
        fixed_s=0.99,
    )
    time_end = time.time()
    matches = [np.arange(pis[0].shape[0]), np.argmax(pis[0], axis=1), np.sum(pis[0], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                     | 0/12 [00:00<?, ?it/s]/home/ylu/project/Benchmark_alignment_methods/notebooks/MOSTA/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|2.248755e+00|0.000000e+00|0.000000e+00
    1|2.059733e+00|9.176979e-02|1.890213e-01
    2|2.003313e+00|2.816357e-02|5.642044e-02
    3|2.001354e+00|9.787676e-04|1.958860e-03
